In [ ]:
import pandas as pd

from pathlib import Path
from pycparser import c_parser

### 编译c源码为ast

In [ ]:
parser = c_parser.CParser()

root = 'data/'
language = 'c/'
source = pd.read_pickle(root+language+'programs.pkl')
#print(source)
# label是指该段源代码来自于哪个类别吗？
source.columns = ['id', 'code', 'label']
print('parse starting')
source['code'] = source['code'].apply(parser.parse)
source.to_pickle(root+'c/ast.pkl')
print('parse ending')
#source_code = source['code'].iloc[0]
#print(type(source_code))
#ast= parser.parse(source_code)
#ast.show()

### 编译java源码为ast

In [12]:
import javalang
def parse_program(func):
    tokens = javalang.tokenizer.tokenize(func)
    #print(f'tokens化得到的序列:{tokens}')
    parser = javalang.parser.Parser(tokens)
    tree = parser.parse_member_declaration()
    return tree



In [13]:
from pycparser import c_parser

import pandas as pd

root = 'data/'
language = 'java/'
source_java = pd.read_csv(root+language+'bcb_funcs_all.tsv',
                            sep='\t', 
                            header=None, 
                            encoding='utf-8')
source_java.columns = ['id', 'code']
print('parse starting')
source_java['code'] = source_java['code'].apply(parse_program)
source_java.to_pickle('data/java/ast.pkl')
print('parse ending')
#java = source_java['code'].iloc[2]
#print(java)
#source['code'] = source['code'].apply(parse_program)



parse starting
parse ending


### 分割pairs 为训练集、验证集、测试集

In [5]:
from pathlib import Path

def split_data(root = 'data/', ratio = '3:1:1', language = 'c'):
  if language is 'c':
    filename = 'oj_clone_ids.pkl'
  else:
    filename = 'bcb_pair_ids.pkl'
  path = root + language + '/' + filename
  data = pd.read_pickle(path)
  data_num = len(data)
  ratios = [int(r) for r in ratio.split(':')]
  train_split = int(ratios[0]/sum(ratios)*data_num)
  val_split = train_split + int(ratios[1]/sum(ratios)*data_num)
  
  #对要分割的数据进行随机打散
  print('starting split')
  data = data.sample(frac=1, random_state=666)

  train = data.iloc[:train_split]
  dev = data.iloc[train_split:val_split]
  test = data.iloc[val_split:]

  def check_or_create(path):
    if not os.path.exists(path):
      os.mkdir(path) 
  train_path = root+language+'/train/'
  check_or_create(train_path)
  assert Path(train_path).is_dir()
  train_file_path = train_path + 'train.pkl'
  train.to_pickle(train_file_path)

  dev_path = root+language+'/dev/'
  check_or_create(dev_path)
  dev_file_path = dev_path + 'dev.pkl'
  train.to_pickle(dev_file_path)

  test_path = root+language+'/test/'
  check_or_create(test_path)
  test_file_path = test_path + 'test.pkl'
  test.to_pickle(test_file_path)
  print('end split')



In [6]:
split_data(language='c')
split_data(language = 'java')

starting split
end split
starting split
end split


### 路径设置

In [18]:
language = 'c'


### 读取c源码训练集所有的ast

In [19]:
import pandas as pd
def read_ast(root = 'data/',language='c'):
  source = pd.read_pickle(root+language+'/ast.pkl')
  print(len(source['code']))
  return source
  # ast = source['code'].iloc[1]
  # ast.show()

In [20]:
#source = read_ast('c')
source_c_ast_DF = read_ast(language='c')
#source_java_ast_DF = read_ast('java')

52001


In [22]:
root = 'data/'
language = 'c'
train_file_path = root + language + '/train/train.pkl'
pairs = pd.read_pickle(train_file_path)
# 找到训练集所包含的所有的source code id
train_ids = pairs['id1'].append(pairs['id2']).unique()
print(train_ids)

[9566 8200 3897 ... 9254 3971 39727]


In [23]:
sys.path.append('..')
from tree import ASTNode, SingleNode 

### 先序遍历得到ast所有的节点（为了进行word2vec训练）

In [12]:
# func函数每次处理ast的当前遍历到的结点。即剩余子树的根节点
def func(root,sequence):
  #print(f'c: partial ast root: {root.__class__.__name__}')
  current = SingleNode(root)
  sequence.append(current.get_token())
  #print(f'sequence: {sequence}')
  for _, child in root.children():
      func(child, sequence)
  if current.get_token().lower() == 'compound':
      sequence.append('End')



In [13]:
def trans_to_sequences(ast):
    sequence = []
    func(ast, sequence)
    return sequence

In [24]:
# 设置ast_DFs的索引为id
ast_copy = source_c_ast_DF.copy(False)

# 得到训练集所包含的ast
ast_DFs = ast_copy.set_index('id',drop=False).loc[train_ids]
#print(ast_DFs)

asts = ast_DFs['code']
sequence = []
corpus = asts.apply(trans_to_sequences)
print(corpus)

#print(type(corpus)) # Series
str_corpus = [' '.join(c) for c in corpus]

# 保存遍历得到的序列到文件
#ast_seq = source_c_ast_DF.copy(False)
#ast_seq['code'] = pd.Series(str_corpus)
#ast_seq.to_pickle(root+language+'/train/ast_seq.pkl')

NameError: name 'trans_to_sequences' is not defined

### 使用训练集训练一个词典

In [16]:
# 使用word2vec从训练集中训练一个词典
from gensim.models.word2vec import Word2Vec
embedding_size = 128
w2v = Word2Vec(corpus, size=embedding_size, workers=16, sg=1, max_final_vocab=3000)
w2v.save(root+language+'/train/embedding_astnode_w2v_' + str(embedding_size))

# generate block sequences with index representations
# 划分语句树（block），并使用训练好的词典初始化语句树的节点

In [25]:
sys.path.append('..')
from prepare_data import get_blocks

In [31]:
root = 'data/'
language = 'c'
embedding_size = 128

In [27]:
def tree_to_index(node):
        token = node.token
        result = [vocab[token].index if token in vocab else max_token]
        children = node.children
        for child in children:
            #print(f'tree_to_index: a node''s child: {child.token}')
            result.append(tree_to_index(child))
        return result

In [28]:
def trans2seq(ast):
        blocks = []
        get_blocks(ast, blocks)
        tree = []
        for b in blocks:
            btree = tree_to_index(b)
            tree.append(btree)
        return tree

In [29]:
# generate block sequences with index representations
def generate_block_seqs(asts):
    # 载入词典
    word2vec = Word2Vec.load(root+language+'/train/embedding_astnode_w2v_' + str(embedding_size)).wv
    vocab = word2vec.vocab
    max_token = word2vec.syn0.shape[0]
    
    # source_c_ast_DF就是id code(ast形式) label 
    source_2_seq_DF =  asts.copy(False)
    trees = source_2_seq_DF['code']
    # 确认trees copy成功
    #print(trees)
    # trans2seq操作是在ast上进行的
    # 把每一个ast转换成seq blocks
    source_2_seq_DF['code'] = trees.apply(trans2seq)

    if 'label' in source_2_seq_DF.columns:
        source_2_seq_DF.drop('label', axis=1, inplace=True)

    blocks = source_2_seq_DF
    return blocks


In [30]:
trees = generate_block_seqs(source_c_ast_DF)

FileNotFoundError: [Errno 2] No such file or directory: 'datac/train/embedding_astnode_w2v_128'

In [ ]:
def merge(data_path,part):
    pairs = pd.read_pickle(data_path)
    pairs['id1'] = pairs['id1'].astype(int)
    pairs['id2'] = pairs['id2'].astype(int)
    df = pd.merge(pairs, blocks, how='left', left_on='id1', right_on='id')
    df = pd.merge(df, blocks, how='left', left_on='id2', right_on='id')
    df.drop(['id_x', 'id_y'], axis=1,inplace=True)
    df.dropna(inplace=True)

    df.to_pickle(root+language+'/'+part+'/blocks.pkl')

In [ ]:
language = 'c'
train_file_path = 'clone/data/'+language+'/train/train.pkl'
dev_file_path = 'clone/data/'+language+'/dev/train.pkl'
test_file_path = 'clone/data/'+language+'/test/train.pkl'

In [ ]:
merge(train_file_path, 'train')
merge(dev_file_path, 'dev')
merge(test_file_path, 'test')

In [3]:
lens = [len(item) for item in tree]
        max_len = max(lens)

IndentationError: unexpected indent (<ipython-input-3-d6d65541cb23>, line 2)